# Generating Populations and Species
Assuming that the civilization is on the verge of moving from a phaze 0 to phase 1 civ.
* baseline species
* n populations (pops)
* population varience (within the norms of the species)
* allegiance groups to factions

Starting with an example user form data

In [1]:

from numpy import random, round, interp, linspace
import pickle
from sklearn.cluster import KMeans
import altair as alt
import sys, os
from sklearn.cluster import KMeans

import pandas as pd


origional `data` comes from the user via the form in `forms.py`

In [2]:
starting_attributes = ['conformity', 'literacy', 'aggression', 'constitution']

data = {
    'conformity':0.4,
    'literacy':0.7,
    'aggression':0.5,
    'constitution':0.4,
    'starting_pop': 100
}

def build_species(data):
    species = {}
    for attr in starting_attributes:
        species[attr] = data[attr]
    return species
    
species = build_species(data)
species


{'conformity': 0.4, 'literacy': 0.7, 'aggression': 0.5, 'constitution': 0.4}

In [3]:
sys.path.append('..')
import helpers.dbquery as db
import helpers.functions as f

## Creating groups of populations, all with different attributes

Populations vary based on conformity, some populations are more literate, some are more aggressive. 

In [4]:
pop_std = .2 * (1-species['conformity'])
print("the population conformity: ", pop_std)

def vary_pops(species):
    pop = {}
    for k in list(species.keys()):
        pop[k] = abs(round(random.normal(species[k], pop_std),3))
    return pop

pops = pd.DataFrame([vary_pops(species) for i in range(data['starting_pop'])])
pops

the population conformity:  0.12


,conformity,literacy,aggression,constitution
0,0.415,0.579,0.445,0.426
1,0.252,0.519,0.742,0.452
2,0.514,0.583,0.431,0.477
3,0.374,0.617,0.519,0.569
4,0.439,0.670,0.350,0.373
...,...,...,...,...
95,0.428,0.759,0.311,0.382
96,0.498,0.697,0.346,0.350
97,0.310,0.754,0.452,0.588
98,0.505,0.724,0.525,0.326


The end result is that population factions are grouped by the idealogical differences of the people in them. This gives each faction a distinct culture. 

## Dividing pops into factions

I feel like there isn't a way to pick an ideal number of factions procedurally,  so until I think of something better I'm just going to go with an arbitrary range based on `population_conformity`.

The amount of different nations is relative to `1-population_conformity` and then scaled out over a number of steps defined as `n_steps`

In [5]:
n_steps = 6

def get_n_factions(n_steps):
    x = interp(
        (1-data["conformity"]),
            linspace(0, 1, num=n_steps),
            [i for i in range(n_steps)]
        )
    return int(round(x))

get_n_factions(n_steps)

3

In [6]:
kmeans = KMeans(n_clusters=n_steps).fit(pops[starting_attributes])
pops['faction_no'] = kmeans.labels_

In [13]:
factions = pd.DataFrame(kmeans.cluster_centers_, columns=starting_attributes)
factions['name'] = factions['conformity'].apply(lambda x: f.make_word(1))
factions

,conformity,literacy,aggression,constitution,name
0,0.214714,0.662214,0.513643,0.268500,Chak
1,0.341929,0.846000,0.522500,0.497143,Groes
2,0.541067,0.680067,0.565333,0.320267,Rovsk
3,0.408500,0.809187,0.333125,0.341625,Hol
4,0.391167,0.682389,0.667889,0.474944,Lyands
5,0.432217,0.596826,0.414478,0.466478,Evsk


So individual `pops` vary slightly, relative to the `conformity`. And `conformity` also determines the number of factions in the group. 

In [8]:
chart = alt.Chart(pops).mark_circle().encode(x='literacy',y='aggression',color='faction_no:N')
chart

alt.Chart(...)

The cluster centers can represent the 'zeitgeist' of that faction's culture. 

# Other population attributes

## Other attributes that get added
These items aren't used to calculate factions as they refer to resources, not values. These are calculated at the creation of a `pop` and later can be updated. 

* Industry = aggression + constitution


In [16]:
pops['industry'] = (pops['aggression'] + pops['constitution'])/2
pops

,conformity,literacy,aggression,constitution,faction_no,industry
0,0.415,0.579,0.445,0.426,5,0.4355
1,0.252,0.519,0.742,0.452,4,0.5970
2,0.514,0.583,0.431,0.477,5,0.4540
3,0.374,0.617,0.519,0.569,5,0.5440
4,0.439,0.670,0.350,0.373,5,0.3615
...,...,...,...,...,...,...
95,0.428,0.759,0.311,0.382,3,0.3465
96,0.498,0.697,0.346,0.350,3,0.3480
97,0.310,0.754,0.452,0.588,1,0.5200
98,0.505,0.724,0.525,0.326,2,0.4255
